In [3]:
import torch.utils as utils

# torch.utils.data.DataLoader()

```python
torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    sampler=None,
    batch_sampler=None,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
    drop_last=False,
    timeout=0,
    worker_init_fn=None,
    multiprocessing_context=None,
)
```

**Docstrings**

`DataLoader`集成了数据集和采样器，同时确保了给定数据集的可迭代性，`DataLoader`支持 map-style 和 iterable-style 的数据集，并支持单进程或多进程加载、customizing loading order and optional automatic batching(collation) 和 memory pinning

详见`torch.utils.data`模块的说明文档

个人解读：如其名称，数据加载器，能够在神经网络模型中各种数据加载的操作

**Args**

- dataset: 所提供的数据集

- batch_size: 每一batch加载的数据量

- shuffle: 每一 epoch 是否 shuffle

- sampler: 定义从数据集采样的策略，若此参数未指明，则`shuffle`必须为``False``.

- batch_sampler: 类似`sampler`，单一次返回一batch的索引，与`batch_size`, `shuffle`, `sampler`, `drop_last`互斥

- num_workers: 用于定义数据由多少子进程加载；0表示数据集由主进程加载

- collate_fn (callable): 将样本合并形成一个小批tensor，常在使用批处理加载 map-style 的数据集时使用

- pin_memory: ``True``时 data loader会将 tensor 复制到 CUDA 的固定内存计算，随后再返回计算值；如果数据元素是自定义类型，或者`collate_fn`返回的批处理是自定义类型，请参见下面的示例（然而说明文档后面没有写示例）

- drop_last: ``True``时将最后不完整的 batch 丢掉

- timeout: if positive, the timeout value for collecting a batch from workers. Should always be non-negative. (default: ``0``)

- worker_init_fn (callable): 若不为``None``，则会在每一个子进程的 seeding 之后、数据加载之前，通过进程的 id 调用该函数

- warning: If the ``spawn`` start method is used, `worker_init_fn` cannot be an unpicklable object, e.g., a lambda function. See `multiprocessing-best-practices` on more details related to multiprocessing in PyTorch.

- note: ``len(dataloader)`` heuristic is based on the length of the sampler used. When `dataset` is an `~torch.utils.data.IterableDataset`, ``len(dataset)`` (if implemented) is returned instead, regardless of multi-process loading configurations, because PyTorch trust user `dataset` code in correctly handling multi-process loading to avoid duplicate data. See `Dataset Types`_ for more details on these two types of datasets and how `~torch.utils.data.IterableDataset` interacts with `Multi-process data loading`_.

#  

#  

# utils.data.Dataset()
`utils.data.Dataset()`

**Docstring**

表示一个`Dataset`的抽象类，所有表示从键到数据的映射的数据集都应继承它；所有子类都应重写`__getitem__`方法以支持根据给定的键来获取数据，并根据需要重写`__len__`方法，该方法常由`torch.utils.data.Sampler`、`torch.utils.data.DataLoader`等实例对象调用以获得数据集的大小；注意，`torch.utils.data.DataLoader`默认构造一个索引采样器用于生成整数型索引，为使该子类能够以映射式数据集的方式工作，需提供自定义采样器

**File**:         \torch\utils\data\dataset.py

**Type**:           type

**Subclasses**:     IterableDataset, TensorDataset, ConcatDataset, Subset

In [4]:
utils.data.random_split()

Help on package torch.utils.data in torch.utils:

NAME
    torch.utils.data

PACKAGE CONTENTS
    _utils (package)
    dataloader
    dataset
    distributed
    sampler

FILE
    d:\programmefiles\python\anaconda3\envs\tensorflow2.2\lib\site-packages\torch\utils\data\__init__.py


